In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


In [2]:
# This is needed to display the images.
%matplotlib inline

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

ModuleNotFoundError: No module named 'object_detection'

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

import cv2
cap = cv2.VideoCapture(0)

# IMPORTANT : this file was found on https://pythonprogramming.net/video-tensorflow-object-detection-api-tutorial/?completed=/introduction-use-tensorflow-object-detection-api-tutorial/
# and updated by Lambert Rosique for PenseeArtificielle.fr for the sole purpose of a tutorial.

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")


# ## Object detection imports
# Here are the imports from the object detection module.

# In[3]:

#from utils import label_map_util
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/home/mahim/Desktop/Nasim/models/research/object_detection/utils/visualization_utils.py:29: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/mahim/anaconda3/envs/workshop/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mahim/anaconda3/envs/workshop/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mahim/anaconda3/envs/workshop/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mahim/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mahim/anaconda3/envs/workshop/lib/pytho

In [2]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90


# ## Download Model

# In[5]:

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

In [4]:
PATH_TO_LABELS = os.path.join('mscoco_label_map.pbtxt')
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


# ## Loading label map
# Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

# In[7]:

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


# ## Helper code

# In[8]:

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


In [6]:
import time
cap = cv2.VideoCapture("test.mp4")
out = None

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

with detection_graph.as_default():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(graph=detection_graph, config=config) as sess:
        frame_num = 1490;
        while frame_num:
            
            frame_num -= 1
            ret, image = cap.read()
            if ret == 0:
                break
            if out is None:
                [h, w] = image.shape[:2]
                out = cv2.VideoWriter("test_out.avi", 0, 25.0, (w, h))
            image_np = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

              # the array based representation of the image will be used later in order to prepare the
              # result image with boxes and labels on it.
              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
              # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
              # Each score represent how level of confidence for each of the objects.
              # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
              # Actual detection.
            start_time = time.time()
            (boxes, scores, classes, num_detections) = sess.run(
                  [boxes, scores, classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
            elapsed_time = time.time() - start_time
            print('inference time cost: {}'.format(elapsed_time))
              #print(boxes.shape, boxes)
              #print(scores.shape,scores)
              #print(classes.shape,classes)
              #print(num_detections)
              # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
        #          image_np,
                  image,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  line_thickness=4)
            out.write(image)
        cap.release()
        out.release()

inference time cost: 14.572384119033813
inference time cost: 0.16298675537109375
inference time cost: 0.14736342430114746
inference time cost: 0.14128422737121582
inference time cost: 0.13771319389343262
inference time cost: 0.13500452041625977
inference time cost: 0.13116097450256348
inference time cost: 0.13944172859191895
inference time cost: 0.14225554466247559
inference time cost: 0.15691041946411133
inference time cost: 2.4533374309539795
inference time cost: 0.13303279876708984
inference time cost: 0.1351315975189209
inference time cost: 0.13402867317199707
inference time cost: 0.13666319847106934
inference time cost: 0.13245606422424316
inference time cost: 0.1338672637939453
inference time cost: 0.13906288146972656
inference time cost: 0.13165020942687988
inference time cost: 0.13578176498413086
inference time cost: 0.13203096389770508
inference time cost: 0.13681340217590332
inference time cost: 0.13300871849060059
inference time cost: 0.1420443058013916
inference time cost: 

inference time cost: 0.14088702201843262
inference time cost: 0.13997650146484375
inference time cost: 0.13003754615783691
inference time cost: 0.13141512870788574
inference time cost: 0.12964797019958496
inference time cost: 0.12920522689819336
inference time cost: 0.15358281135559082
inference time cost: 0.13495302200317383
inference time cost: 0.13378381729125977
inference time cost: 0.12959980964660645
inference time cost: 0.13956499099731445
inference time cost: 0.13827848434448242
inference time cost: 0.13372302055358887
inference time cost: 0.13623356819152832
inference time cost: 0.13323330879211426
inference time cost: 0.12979626655578613
inference time cost: 0.12868142127990723
inference time cost: 0.13436365127563477
inference time cost: 0.1669154167175293
inference time cost: 0.13006114959716797
inference time cost: 0.1429598331451416
inference time cost: 0.1324901580810547
inference time cost: 0.138092041015625
inference time cost: 0.12776470184326172
inference time cost: 

inference time cost: 0.13977789878845215
inference time cost: 0.13286542892456055
inference time cost: 0.133774995803833
inference time cost: 0.13191580772399902
inference time cost: 0.1367650032043457
inference time cost: 0.19060277938842773
inference time cost: 0.1307063102722168
inference time cost: 0.13922452926635742
inference time cost: 0.1342155933380127
inference time cost: 0.14939379692077637
inference time cost: 0.14333534240722656
inference time cost: 0.13806557655334473
inference time cost: 0.13821005821228027
inference time cost: 0.1326897144317627
inference time cost: 0.1320054531097412
inference time cost: 0.13421010971069336
inference time cost: 0.13217902183532715
inference time cost: 0.15731167793273926
inference time cost: 0.1329021453857422
inference time cost: 0.14257073402404785
inference time cost: 0.13492512702941895
inference time cost: 0.13224363327026367
inference time cost: 0.13138413429260254
inference time cost: 0.13994526863098145
inference time cost: 0.1